In [1]:
import pandas as pd, re, nltk, numpy as np, string
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
german_stop_words = stopwords.words('german')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dorian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dorian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df = pd.read_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/data_train.csv', encoding='utf-8')
df.head()

,source,summary
0,Minghella war der Sohn italienisch-schottische...,"Anthony Minghella, CBE war ein britischer Film..."
1,Ende der 1940er Jahre wurde eine erste Auteur-...,Die Auteur-Theorie ist eine Filmtheorie und di...
2,"Al Pacino, geboren in Manhattan, ist der Sohn ...","Alfredo James ""Al"" Pacino ist ein US-amerikani..."
3,Der Name der Alkalimetalle leitet sich von dem...,Als Alkalimetalle werden die chemischen Elemen...
4,Die Arbeit ist bereits seit dem Altertum Gegen...,Das deutsche Arbeitsrecht ist ein Rechtsgebiet...


In [2]:
df = pd.read_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/data_class.csv', encoding='utf-8')

In [8]:
df = df.drop(['source', 'summary', 'keywords'], axis=1)

In [33]:
def calc_importance(df):
    german_stop_words = stopwords.words('german')
    TfidfVec = TfidfVectorizer()
    tokens = sent_tokenize(df, language="german")
    if TfidfVec.fit_transform(tokens) == NaN:
        print(9)
    else:    
        print (TfidfVec.fit_transform(tokens))
    #return TfidfVec.fit_transform(df)

df.source1 = df.source1.apply(calc_importance)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [12]:
stopword_list = nltk.corpus.stopwords.words('german')

def remove_stopwords(text):
    filtered_words = [word for word in nltk.word_tokenize(text) if word not in stopword_list]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

df["source"] = df["source"].apply(remove_stopwords)

KeyError: 'source'

In [6]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [7]:
from nltk.corpus import stopwords
german_stop_words = stopwords.words('german')

cv = CountVectorizer(max_df=0.85,stop_words=german_stop_words, max_features=10000)
word_count_vector = cv.fit_transform(df['source'])

In [8]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [9]:
tfidf_transformer.idf_
feature_names=cv.get_feature_names()

In [10]:
#doc = df['source'][2568]
def get_kew(df):
    kw=[]
    tf_idf_vector=tfidf_transformer.transform(cv.transform([df]))
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    keywords = extract_topn_from_vector(feature_names,sorted_items,10)
    for k in keywords:
        kw.append(k)
    return kw

df['keywords'] = df['source'].apply(get_kew)

In [ ]:
df.head()

In [11]:
import numpy as np
import pandas as pd

# Getting s as pandas series which has split on full stop and new sentence a new line         
s = df["source"].apply(lambda x : sent_tokenize(x)).apply(pd.Series,1).stack()
s.index = s.index.droplevel(-1) # to line up with df's index
s.name = 'source' # needs a name to join

x=s.to_frame(name='source1')

# Joining should ideally get me proper output. But I am getting original dataframe instead of split one.
df = df.join(x)

In [12]:
#@title Default title text
def CapWords(df):
    return sum(1 for c in df if c.isupper())

def W_sourceSummary(df, df2):
    text2 = df2
    text2 = text2.translate(str.maketrans('', '', string.punctuation))
    tokens = sent_tokenize(df, language = "german")
    tok2 = word_tokenize(text2, language = "german")
    for s in tokens:
      #print (s)
      return len(set(s.split()).intersection(set(tok2)))

def W_sourceKeywords(df, df2):
    df.lower()
    text2 = df2
    text2 = text2.translate(str.maketrans('', '', string.punctuation))
    tokens = sent_tokenize(df, language = "german")
    tok2 = word_tokenize(text2, language = "german")
    for s in tokens:
        return len(set(s.split()).intersection(set(tok2)))

def listToString(df):
    string = ' '.join([str(e) for e in df])
    return string

In [13]:
df['no_words_inSent_SS'] = df.apply(lambda x: W_sourceSummary(x['source1'], x['summary']), axis=1)
df['keywords'] = df['keywords'].apply(listToString)
df['no_words_inSent_SK'] = df.apply(lambda x: W_sourceKeywords(x['source1'], x['keywords']), axis=1)
df['noCap_LetterWords_inSentence'] = df['source1'].apply(CapWords)

In [ ]:
german_stop_words = stopwords.words('german')
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

from sklearn.feature_extraction.text import TfidfVectorizer
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=german_stop_words)
def cos_similarity(textlist):
    tfidf = TfidfVec.fit_transform(textlist)
    return (tfidf * tfidf.T).toarray()

df["source"] = df.apply(cos_similarity(df.source1))

In [20]:
nltk.download('wordnet') # first-time use only
german_stop_words = stopwords.words('german')
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

from sklearn.feature_extraction.text import TfidfVectorizer
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=german_stop_words)
def cos_similarity(textlist):
    if TfidfVec.fit_transform(textlist)==None:
        return 0
#        continue
    else:
        tfidf = TfidfVec.fit_transform(textlist)
        return (tfidf * tfidf.T).toarray()

def get_docs(df):
    threshold = 0.15
    doc = sent_tokenize(df, language="german")
    lst, output_tuples, similar_sent = [], [], [] 
    arr = cos_similarity(doc)
    print (arr)
    doc_names = ["s"+str(s) for s in range(0, len(doc), 1)]
    for row in range(len(arr)):
        lst = [row+1+idx for idx, num in \
                  enumerate(arr[row, row+1:]) if num >= threshold]
        [(output_tuples.append((doc[row], doc[item]))) for item in lst]
    for tup in output_tuples:
        for sent in tup:
            similar_sent.append(sent)
    for sent in doc:
        if sent in set(doc).intersection(set(similar_sent)):
            return 1
        else:
            return 0

#get_docs(df['source'][0])
df['sim_sent'] = df['source1'].apply(get_docs)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dorian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.        0.0724038]
 [0.0724038 1.       ]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[

[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]

[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1. 0.]
 [0. 1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1. 0.]
 [0. 1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [23]:
df.head()
df.drop(['source', 'summary', 'keywords'], axis=1)

,source1,no_words_inSent_SS,no_words_inSent_SK,noCap_LetterWords_inSentence
0,Minghella Sohn italienisch-schottischer Eltern...,1,0,7
0,Nach Schulabschluss studierte Universität Hul...,0,0,6
0,1978 drehte ersten Kurzfilm .,0,0,1
0,Seit 1981 Autor Story Editor tätig .,0,0,4
0,"Er wurde Theaterstücken , Rundfunkhörspielen...",0,0,9
...,...,...,...,...
99999,Carl Sigmans Text erschien 1960er Jahren briti...,0,0,7
99999,Camillo Felgen alias Heinz Helmer verfasste de...,5,1,12
99999,Zahlreiche Interpreten sangen 1964 Lied franzo...,1,1,11
99999,Von Letzterer stammt spanischsprachige Fassung...,2,1,6


In [70]:
df3.to_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/data_class.csv')

In [54]:
df2 = pd.read_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/data_features.csv', encoding='utf-8')

In [57]:
N = 18310
df2 = df2.head(df2.shape[0] -N)

In [62]:
df3 = pd.concat([df.reset_index(drop=True),df2.reset_index(drop=True)], axis=1)

In [63]:
df3

,Unnamed: 0,source1,no_words_inSent_SS,no_words_inSent_SK,noCap_LetterWords_inSentence,sim_sent
0,0,Minghella Sohn italienisch-schottischer Eltern...,1,0,7,[1
1,0,Nach Schulabschluss studierte Universität Hul...,0,0,6,0
2,0,1978 drehte ersten Kurzfilm .,0,0,1,0
3,0,Seit 1981 Autor Story Editor tätig .,0,0,4,0
4,0,"Er wurde Theaterstücken , Rundfunkhörspielen...",0,0,9,0
...,...,...,...,...,...,...
3513930,99999,Carl Sigmans Text erschien 1960er Jahren briti...,0,0,7,1
3513931,99999,Camillo Felgen alias Heinz Helmer verfasste de...,5,1,12,0
3513932,99999,Zahlreiche Interpreten sangen 1964 Lied franzo...,1,1,11,1
3513933,99999,Von Letzterer stammt spanischsprachige Fassung...,2,1,6,0


In [67]:
import re
def clean(df):
    if re.search('\[.*', df):
       return re.sub('\[', '', df)
    elif  re.search('.*\].*', df):
        return re.sub('\]', '', df)
    else:
        return df

df3.sim_sent = df3.sim_sent.apply(clean)

In [72]:
df3.

,Unnamed: 0,source1,no_words_inSent_SS,no_words_inSent_SK,noCap_LetterWords_inSentence,sim_sent
0,0,Minghella Sohn italienisch-schottischer Eltern...,1,0,7,1
1,0,Nach Schulabschluss studierte Universität Hul...,0,0,6,0
2,0,1978 drehte ersten Kurzfilm .,0,0,1,0
3,0,Seit 1981 Autor Story Editor tätig .,0,0,4,0
4,0,"Er wurde Theaterstücken , Rundfunkhörspielen...",0,0,9,0
...,...,...,...,...,...,...
3513930,99999,Carl Sigmans Text erschien 1960er Jahren briti...,0,0,7,1
3513931,99999,Camillo Felgen alias Heinz Helmer verfasste de...,5,1,12,0
3513932,99999,Zahlreiche Interpreten sangen 1964 Lied franzo...,1,1,11,1
3513933,99999,Von Letzterer stammt spanischsprachige Fassung...,2,1,6,0


In [ ]:
df2.set_index(['source']).sim_sent.apply(pd.Series).stack()\
.reset_index(name = 'sim_sent').drop('level_2', axis = 1)

In [55]:
df2 = df2.assign(sim_sent=df2.sim_sent.str.split(",")).explode('sim_sent')

In [56]:
df2 = df2[['sim_sent']]

In [50]:
df3 = df.assign(sim_sent=df2.sim_sent.str.split(",")).explode('sim_sent')

pd.concat([df,df3], axis=0, ignore_index=True)

,source,summary,keywords,source1,no_words_inSent_SS,no_words_inSent_SK,noCap_LetterWords_inSentence,sim_sent,Unnamed: 0
0,Minghella Sohn italienisch-schottischer Eltern...,"Anthony Minghella, CBE war ein britischer Film...",oscar regie opera bbc film bestes fernsehserie...,Minghella Sohn italienisch-schottischer Eltern...,1,0,7,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN
1,Minghella Sohn italienisch-schottischer Eltern...,"Anthony Minghella, CBE war ein britischer Film...",oscar regie opera bbc film bestes fernsehserie...,Nach Schulabschluss studierte Universität Hul...,0,0,6,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN
2,Minghella Sohn italienisch-schottischer Eltern...,"Anthony Minghella, CBE war ein britischer Film...",oscar regie opera bbc film bestes fernsehserie...,1978 drehte ersten Kurzfilm .,0,0,1,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN
3,Minghella Sohn italienisch-schottischer Eltern...,"Anthony Minghella, CBE war ein britischer Film...",oscar regie opera bbc film bestes fernsehserie...,Seit 1981 Autor Story Editor tätig .,0,0,4,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN
4,Minghella Sohn italienisch-schottischer Eltern...,"Anthony Minghella, CBE war ein britischer Film...",oscar regie opera bbc film bestes fernsehserie...,"Er wurde Theaterstücken , Rundfunkhörspielen...",0,0,9,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN
...,...,...,...,...,...,...,...,...,...
7046175,Umberto Bindi 1960er-Jahren bekanntesten Vertr...,NaN,NaN,NaN,"[2, 2, 6, 6, 2, 0, 5, 2, 6, 0, 2, 0, 0, 0, 6, ...","[0, 0, 1, 1, 0, 0, 4, 1, 1, 0, 1, 0, 0, 0, 1, ...","[6, 7, 14, 7, 10, 9, 15, 18, 14, 1, 8, 1, 2, 1...",0,99999.0
7046176,Umberto Bindi 1960er-Jahren bekanntesten Vertr...,NaN,NaN,NaN,"[2, 2, 6, 6, 2, 0, 5, 2, 6, 0, 2, 0, 0, 0, 6, ...","[0, 0, 1, 1, 0, 0, 4, 1, 1, 0, 1, 0, 0, 0, 1, ...","[6, 7, 14, 7, 10, 9, 15, 18, 14, 1, 8, 1, 2, 1...",1,99999.0
7046177,Umberto Bindi 1960er-Jahren bekanntesten Vertr...,NaN,NaN,NaN,"[2, 2, 6, 6, 2, 0, 5, 2, 6, 0, 2, 0, 0, 0, 6, ...","[0, 0, 1, 1, 0, 0, 4, 1, 1, 0, 1, 0, 0, 0, 1, ...","[6, 7, 14, 7, 10, 9, 15, 18, 14, 1, 8, 1, 2, 1...",0,99999.0
7046178,Umberto Bindi 1960er-Jahren bekanntesten Vertr...,NaN,NaN,NaN,"[2, 2, 6, 6, 2, 0, 5, 2, 6, 0, 2, 0, 0, 0, 6, ...","[0, 0, 1, 1, 0, 0, 4, 1, 1, 0, 1, 0, 0, 0, 1, ...","[6, 7, 14, 7, 10, 9, 15, 18, 14, 1, 8, 1, 2, 1...",0,99999.0


In [51]:
df

,source,summary,keywords,source1,no_words_inSent_SS,no_words_inSent_SK,noCap_LetterWords_inSentence,sim_sent
0,Minghella Sohn italienisch-schottischer Eltern...,"Anthony Minghella, CBE war ein britischer Film...",oscar regie opera bbc film bestes fernsehserie...,Minghella Sohn italienisch-schottischer Eltern...,1,0,7,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
0,Minghella Sohn italienisch-schottischer Eltern...,"Anthony Minghella, CBE war ein britischer Film...",oscar regie opera bbc film bestes fernsehserie...,Nach Schulabschluss studierte Universität Hul...,0,0,6,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
0,Minghella Sohn italienisch-schottischer Eltern...,"Anthony Minghella, CBE war ein britischer Film...",oscar regie opera bbc film bestes fernsehserie...,1978 drehte ersten Kurzfilm .,0,0,1,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
0,Minghella Sohn italienisch-schottischer Eltern...,"Anthony Minghella, CBE war ein britischer Film...",oscar regie opera bbc film bestes fernsehserie...,Seit 1981 Autor Story Editor tätig .,0,0,4,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
0,Minghella Sohn italienisch-schottischer Eltern...,"Anthony Minghella, CBE war ein britischer Film...",oscar regie opera bbc film bestes fernsehserie...,"Er wurde Theaterstücken , Rundfunkhörspielen...",0,0,9,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...
99999,Umberto Bindi 1960er-Jahren bekanntesten Vertr...,Il mio mondo ist der Titel eines italienischsp...,my you world il re text 1964 mio song lied,Carl Sigmans Text erschien 1960er Jahren briti...,0,0,7,"[1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, ..."
99999,Umberto Bindi 1960er-Jahren bekanntesten Vertr...,Il mio mondo ist der Titel eines italienischsp...,my you world il re text 1964 mio song lied,Camillo Felgen alias Heinz Helmer verfasste de...,5,1,12,"[1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, ..."
99999,Umberto Bindi 1960er-Jahren bekanntesten Vertr...,Il mio mondo ist der Titel eines italienischsp...,my you world il re text 1964 mio song lied,Zahlreiche Interpreten sangen 1964 Lied franzo...,1,1,11,"[1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, ..."
99999,Umberto Bindi 1960er-Jahren bekanntesten Vertr...,Il mio mondo ist der Titel eines italienischsp...,my you world il re text 1964 mio song lied,Von Letzterer stammt spanischsprachige Fassung...,2,1,6,"[1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, ..."


In [ ]:
#df2 = pd.read_csv('/content/sample_data/data_features.csv', encoding='utf-8') 
dat = df.drop(['summary', 'keywords'], axis=1)

In [ ]:
def CapWords(df):
    return sum(1 for c in df if c.isupper())

df_new['noCap_LetterWords_inSentence'] = df_new['source'].apply(CapWords)

In [ ]:
dat.to_csv('/content/sample_data/data_features.csv')

In [ ]:
dat.iloc[7]

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

'''X_train = np.array(sent_tokenize(dat['source'][0], language="german"))
y_train = np.array([[1],[0],[0],[0],[1],[1],[1],[1],[0],[1],[1],[0],[1],[1],[0],[1],[1],[0],[1],[0],[1],[0],[1]])

X_test = np.array(["it's a nice day in nyc",
                   'i loved the time i spent in london, the weather was great, though there was a nip in the air and i had to wear a jacket.'
                   ])  ''' 

X_train,y_train,X_test,y_test = train_test_split(np.array(np.array(sent_tokenize(dat['source'][0], language="german"))),\
                                                 np.array(dat['sim_sent'][0]), test_size=0.3, random_state=25)


def get_text_length(x):
    return np.array([len(t) for t in x]).reshape(-1, 1)

classifier = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=1,max_df=2)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('length', Pipeline([
            ('count', FunctionTransformer(get_text_length, validate=False)),
        ]))
    ])),
    ('clf', RandomForestClassifier(n_estimators = 100))])

classifier.fit(X_train, y_train)
predicted = classifier.predict(X_test)
predicted

In [ ]:
from sklearn.model_selection import train_test_split
d = dat[['no_words_inSent_SS', 'no_words_inSent_SK', 'noCap_LetterWords_inSentence']]
X_train, X_test, Y_train, Y_test = train_test_split(data, dat['sim_sent'],test_size=0.3, random_state=25)